In [54]:
import matplotlib as mpl
import pyvista as pv
import ufl
import numpy as np

from petsc4py import PETSc
from mpi4py import MPI

from dolfinx import fem, mesh, io, plot
from dolfinx.fem.petsc import assemble_vector, assemble_matrix, create_vector, apply_lifting, set_bc

# Define temporal parameters
t = 0  # Start time
T = 10 # Final time
num_steps = 100
dt = T / num_steps  # time step size


# Define mesh
nx, ny = 50, 50
domain = mesh.create_rectangle(
    MPI.COMM_WORLD, [np.array([-5, -5]), np.array([5,-1])], [50, 50], mesh.CellType.triangle
)
V = fem.functionspace(domain, ("Lagrange", 1))

In [55]:
# intial pore pressures * and pattern 
def intial_conditions(x, q=10.0, x0=0.0, z0=-0.001, eps = 1e-3):
    z = np.maximum(-x[1], eps)   # depth measured from y=0 downward
    dx = x[0] - x0
    return (2*q/np.pi) * (z**3) / (dx**2 + z**2)**2

u_n = fem.Function(V)
u_n.name = "u_n"
u_n.interpolate(intial_conditions)

atol = 1e-12
fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(
    domain, dim=fdim,
    marker=lambda x: np.logical_or.reduce([
        np.isclose(x[0], -5.0, atol=atol),
        np.isclose(x[0],  5.0, atol=atol),
        np.isclose(x[1], -5.0, atol=atol),
    ])
)

bc = fem.dirichletbc(PETSc.ScalarType(0), fem.locate_dofs_topological(V, fdim, boundary_facets), V)

In [56]:
xdmf = io.XDMFFile(domain.comm, "consolidation.xdmf", "w")
xdmf.write_mesh(domain)

uh = fem.Function(V)
uh.name = "Pore Pressure"
uh.interpolate(intial_conditions)
xdmf.write_function(uh, t)

In [57]:
u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
f = fem.Constant(domain, PETSc.ScalarType(0))

a = u * v * ufl.dx + dt * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = (u_n + dt * f) * v * ufl.dx

In [58]:
bilinear_form = fem.form(a)
linear_form = fem.form(L)

In [59]:
A = assemble_matrix(bilinear_form, bcs=[bc])
A.assemble()
b = create_vector(linear_form)

In [60]:
solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

In [61]:
grid = pv.UnstructuredGrid(*plot.vtk_mesh(V))

plotter = pv.Plotter()
plotter.open_gif("u_time.gif", fps=10)

# attach your scalar field
grid.point_data["Pore pressure"] = uh.x.array

viridis = mpl.colormaps.get_cmap("viridis").resampled(25)
sargs = dict(title_font_size=25, label_font_size=20, fmt="%.2e", color="black",
             position_x=0.1, position_y=0.8, width=0.8, height=0.1)

# ✅ no more warped mesh — just the flat coloured grid
renderer = plotter.add_mesh(
    grid,
    show_edges=True,
    lighting=False,
    cmap=viridis,
    scalar_bar_args=sargs,
    clim=[0, max(uh.x.array)]
)
plotter.view_xy()


In [62]:
for i in range(num_steps):
    t += dt

    # Update the right-hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    assemble_vector(b, linear_form)

    # Apply Dirichlet boundary condition to the vector
    apply_lifting(b, [bilinear_form], [[bc]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b, [bc])

    # Solve linear problem
    solver.solve(b, uh.x.petsc_vec)
    uh.x.scatter_forward()

    # Update solution at previous time step (u_n)
    u_n.x.array[:] = uh.x.array

    # Write solution to file
    xdmf.write_function(uh, t)

    # ✅ Update plot (no warping — just recolour)
    grid.point_data["uh"] = uh.x.array
    plotter.update_scalars(uh.x.array)
    plotter.write_frame()

# Close files and plotter
plotter.close()
xdmf.close()


In [ ]:
V2 = fem.functionspace(domain, ("Lagrange", 2))
uex = fem.Function(V2)
uex.interpolate(lambda x: (2*q/np.pi) * (z**3) / (dx**2 + z**2)**2)
